In [1]:
import numpy as np
import os
import sys
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from models.CNN import create_cnn_model

In [2]:
def load(f):
    # Move up one directory when loading the data
    file_path = os.path.join('../../../', f)
    return np.load(file_path)['arr_0']

# Load the data
x_train = load('kmnist-train-imgs.npz')/ 255.0
x_test = load('kmnist-test-imgs.npz')/ 255.0
y_train = load('kmnist-train-labels.npz')
y_test = load('kmnist-test-labels.npz')
# Reshape the data for CNN input
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [3]:
input_shape = x_train.shape[1:]  # 784 for KMNIST
num_classes = y_train.max() + 1
x_train_aug, x_val, y_train_aug, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)


In [4]:
hyperparameters = {
    'num_layers': 3,
    'filters': [32, 64, 64],
    'kernel_sizes': [(3, 3), (3, 3), (3, 3)],
    'activations': ['relu', 'relu', 'relu'],
    'dense_units': 64,
    'dense_activation': 'relu',
    'pooling': 'avg',
    'batch_normalization': True
}

In [6]:
best_model_flip = create_cnn_model(input_shape, num_classes, hyperparameters)
best_model_no_flip = create_cnn_model(input_shape, num_classes, hyperparameters)
datagen_flip = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True
)
datagen_no_flip = ImageDataGenerator()
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
datagen_val = ImageDataGenerator()
# Wrap the datagen with our custom generator
train_generator_flip = datagen_flip.flow(x_train_aug, y_train_aug, batch_size=32)
train_generator_no_flip = datagen_no_flip.flow(x_train_aug, y_train_aug, batch_size=32)
val_generator = datagen_val.flow(x_val, y_val, batch_size=32)
# Train the model with custom augmentation
history_flip = best_model_flip.fit(train_generator_flip,
                        epochs=20,
                        validation_data=val_generator,  
                        verbose=1,
                        callbacks=[early_stopping])
history_no_flip = best_model_no_flip.fit(train_generator_no_flip,
                        epochs=20,
                        validation_data=val_generator,
                        verbose=1,
                        callbacks=[early_stopping])

# Evaluate on the test set
test_loss_flip, test_accuracy_flip = best_model_flip.evaluate(x_test, y_test, verbose=0)
test_loss_no_flip, test_accuracy_no_flip = best_model_no_flip.evaluate(x_test, y_test, verbose=0)
print(f"Test accuracy with Flipping: {test_accuracy_flip:.4f}")
print(f"Test loss with Flipping: {test_loss_flip:.4f}")
print(f"Test accuracy without Flipping: {test_accuracy_no_flip:.4f}")
print(f"Test loss without Flipping: {test_loss_no_flip:.4f}")

Epoch 1/20
1688/1688 [==============================] - 12s 6ms/step - loss: 0.4703 - accuracy: 0.8598 - val_loss: 0.3402 - val_accuracy: 0.8907
Epoch 2/20
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2213 - accuracy: 0.9314 - val_loss: 0.2294 - val_accuracy: 0.9295
Epoch 3/20
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1677 - accuracy: 0.9469 - val_loss: 0.1468 - val_accuracy: 0.9542
Epoch 4/20
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1428 - accuracy: 0.9556 - val_loss: 0.2086 - val_accuracy: 0.9358
Epoch 5/20
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1246 - accuracy: 0.9606 - val_loss: 0.1991 - val_accuracy: 0.9362
Epoch 6/20
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1118 - accuracy: 0.9649 - val_loss: 0.1508 - val_accuracy: 0.9548
Epoch 7/20
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1012 - accuracy: 0.9684 - val_loss: 0.1087 - val_accuracy